<a href="https://colab.research.google.com/github/JacobDowns/CSCI-491-591/blob/main/lecture6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced mpi4py Features

* In this lecture we'll cover a handful of advanced features in mpi4py
* We'll discuss parallel I/O, persistent communication, and one sided communication

### Persistent Communication
* In some of examples, particularly the heat equation, we sent many of the same type of message repreatedly in a loop
* In such cases, communication can be optimized by using persistent communication, a particular case of nonblocking communication allowing the reduction of the overhead
* For point-to-point communication, persistent communication is used by setting up requests with `Send_init` and Recv_init`
* In each loop iteration, you would then call `Start` or `Startall` and subsequently `Wait` or `Waitall`



### Usage Pattern
* Create a request one time
```python
req_s = comm.Send_init(buf, dest=..., tag=...)
req_r = comm.Recv_init(buf, source=..., tage=...)
```
* In a loop you can repeat the message with the outlines form many times
```python
req_s.Start()
req_r.Start()
MPI.Request.Waitall([req_s, req_r])
```
After you're finished sending messages, clean up with
```python
req_s.Free()
req_r.Free()
```
* It's a little funky to have to free something in a Python program, but a persistent request creates a `request` that holds onto
  * A pointer to the buffer
  * Datatype description
  * The communicator and tag
* Free will tell MPI you're done with these resources and is another reminder of how MPI is a lower level library being wrapped in Python


### Example: Sending Data in a Ring!
* Below, let's look at an example where we have each rank send some information to its left, wrapping around to the last rank

In [ ]:
from mpi4py import MPI
import numpy as np

comm  = MPI.COMM_WORLD
rank  = comm.Get_rank()
size  = comm.Get_size()

right = (rank + 1) % size
left  = (rank - 1) % size

sendbuf = np.array(rank, dtype='i')   # Will send our rank
recvbuf = np.array(-1, dtype='i')     # Will receive from 'left'

# Build persistent requests once
send_req = comm.Send_init(sendbuf, dest=right, tag=0)
recv_req = comm.Recv_init(recvbuf,  source=left,  tag=0)

n_iters = 5
for it in range(n_iters):
    # Optionally update what we send each iter
    sendbuf[...] = rank + 100*it

    # Start both; then wait for both
    send_req.Start()
    recv_req.Start()
    MPI.Request.Waitall([send_req, recv_req])

    print(f"[iter {it}] rank {rank} got {recvbuf} from {left}")

send_req.Free()
recv_req.Free()